In [80]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [116]:
import pandas as pd

## Connexion a spotify

In [41]:
CRED_PATH_SPOTIFY = "../creditentials-spotify.json"

In [42]:
with open(CRED_PATH_SPOTIFY, 'r') as handle:
    data = json.load(handle)

In [43]:
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    **data
))

## Connexion a google

In [78]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SAMPLE_SPREADSHEET_ID = '1b75J-QTGrujSgF9r0_JPOKkcXAwzFVwpETOAyVBw8ak'
SAMPLE_RANGE_NAME = 'Class Notations!A1:N'

In [59]:
CRED_PATH_GOOGLE = "../credentials-sheets.json"

In [92]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(CRED_PATH_GOOGLE, scope)
client = gspread.authorize(creds)

In [115]:
sh = client.open('Ach! Musik')
sheet = sh.worksheet('Notations')

In [117]:
data = sheet.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)

In [118]:
df

,genre,sub_genre,artist,album,song,ntQ,ntG,ntV,ntR,ntS,ntGl,ntRx,ntC,ntL
0,Pop,Disco,Abba,Abba,"I do, I do, I do, I do, I do",8,"8,5",5,8,,,,,
1,Pop,Disco,Abba,Abba,Mamma Mia,"8,5","8,75","6,5","8,3",,6,,,
2,Pop,Disco,Abba,Abba,SOS,9,9,7,"8,4",,"6,25",,,
3,Pop,Disco,Abba,Arrival,"Knowing me, knowing you","7,5","8,5","5,5","6,5",,,,,
4,Pop,Disco,Abba,Super Trouper,Lay all your love on me,8,"8,5",4,"8,5",,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411,Rock,Punk,The Offspring,Smash,Come Out and Play,8,,,?,,,,,
1412,Rock,Alt Rock,Red Hot Chili Peppers,Californication,Scar Tissue,"8,5",,?,,,,,,
1413,Rock,,The Who,Who's Next,Won't Get Fooled Again,"7,5",,,,,,,,
1414,Rock,Garage Rock,The White Stripes,Elephant,Seven Nation Army,"7,5",,,?,,,,,


In [ ]:
sp.search('cowboys from hell')